# Mosquitos on Human Skin Recognition

In [1]:
# Essential Imports
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Computer Vision Libraries
import cv2
from PIL import Image
import skimage
from skimage import io, transform

# Deep Learning Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms

# TensorFlow/Keras (if needed)
import tensorflow as tf
from tensorflow import keras

# Data Augmentation
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Metrics and Evaluation
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

# Visualization
import plotly.express as px
import plotly.graph_objects as go

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
tf.random.set_seed(42)

print("Environment setup complete!")
print(f"PyTorch version: {torch.__version__}")
print(f"TensorFlow version: {tf.__version__}")
print(f"OpenCV version: {cv2.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

Environment setup complete!
PyTorch version: 2.6.0+cu124
TensorFlow version: 2.19.0
OpenCV version: 4.12.0
CUDA available: True


In [ ]:
# Project Configuration
class Config:
    # Paths
    PROJECT_ROOT = Path.cwd()
    DATA_DIR = PROJECT_ROOT / "data"
    RAW_DATA_DIR = DATA_DIR / "raw"
    PROCESSED_DATA_DIR = DATA_DIR / "processed"
    MODELS_DIR = PROJECT_ROOT / "models"
    OUTPUTS_DIR = PROJECT_ROOT / "outputs"
    LOGS_DIR = PROJECT_ROOT / "logs"
    
    # Model parameters
    IMAGE_SIZE = (224, 224)
    BATCH_SIZE = 32
    LEARNING_RATE = 0.001
    NUM_EPOCHS = 50
    
    # Device: If you want to use your gpu to compute please configure this based on your GPU type
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Create directories if they don't exist
    def create_dirs(self):
        for attr_name in dir(self):
            attr = getattr(self, attr_name)
            if isinstance(attr, Path) and attr_name.endswith('_DIR'):
                attr.mkdir(parents=True, exist_ok=True)
                print(f"Created directory: {attr}")

# Initialize configuration
config = Config()
config.create_dirs()

print(f"Device: {config.DEVICE}")
print(f"Project root: {config.PROJECT_ROOT}")

Created directory: e:\3_Project\COMP9444\COMP_9444_Assignment2\data
Created directory: e:\3_Project\COMP9444\COMP_9444_Assignment2\logs
Created directory: e:\3_Project\COMP9444\COMP_9444_Assignment2\models
Created directory: e:\3_Project\COMP9444\COMP_9444_Assignment2\outputs
Created directory: e:\3_Project\COMP9444\COMP_9444_Assignment2\data\processed
Created directory: e:\3_Project\COMP9444\COMP_9444_Assignment2\data\raw
Device: cpu
Project root: e:\3_Project\COMP9444\COMP_9444_Assignment2


In [7]:
# Utility Functions

def plot_images(images, titles=None, cols=4, figsize=(15, 10)):
    """Plot a grid of images."""
    n_images = len(images)
    rows = (n_images + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    if rows == 1:
        axes = axes.reshape(1, -1)
    
    for i in range(n_images):
        row, col = i // cols, i % cols
        ax = axes[row, col]
        
        if len(images[i].shape) == 3:
            ax.imshow(images[i])
        else:
            ax.imshow(images[i], cmap='gray')
        
        if titles:
            ax.set_title(titles[i])
        ax.axis('off')
    
    # Hide empty subplots
    for i in range(n_images, rows * cols):
        row, col = i // cols, i % cols
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()

def load_image(image_path, target_size=None):
    """Load and optionally resize an image."""
    image = cv2.imread(str(image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    if target_size:
        image = cv2.resize(image, target_size)
    
    return image

def get_model_summary(model, input_size):
    """Get model summary for PyTorch models."""
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Non-trainable parameters: {total_params - trainable_params:,}")

def save_model(model, filepath, metadata=None):
    """Save model with metadata."""
    save_dict = {
        'model_state_dict': model.state_dict(),
        'metadata': metadata or {}
    }
    torch.save(save_dict, filepath)
    print(f"Model saved to: {filepath}")

print("Utility functions loaded successfully!")

Utility functions loaded successfully!
